In [1]:
from config import settings

In [4]:
import requests
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima.arima import ndiffs
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
from keras.models import Sequential
%matplotlib inline

ModuleNotFoundError: No module named 'keras'

In [3]:
from models.data import AlphaVantageAPI
from models.graph import GraphBuilder
from models.model import ArimaModelBuilder, LSTMModelBuilder

ModuleNotFoundError: No module named 'keras'

In [ ]:
av = AlphaVantageAPI()

In [ ]:
ticker = 'IBM'
df = av.get_daily(ticker = ticker)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.asfreq('d')

In [ ]:
df.isnull().sum()

In [ ]:
y = df["close"].fillna(method = "ffill")

In [ ]:
y.isnull().sum()

In [ ]:
y.shape

In [ ]:
y.index

In [ ]:
df_log = np.log(y).dropna()
model_autoARIMA = auto_arima(df_log, start_p=0,
                             start_q=0, start_P=0,
                             start_Q=0, test='adf', error_action='trace')
get_parametes = model_autoARIMA.get_params()
summary = model_autoARIMA.summary()
summary

In [ ]:
df_log.shape

In [ ]:
df_log.index[0]

In [ ]:
test_size = 0.995
limit = int(len(df_log) * test_size)
y_train = df_log.iloc[:limit]
y_test = df_log.iloc[limit:]
len_train = len(y_train)
len_test = len(y_test)
len_train+len_test

In [ ]:
y_test.index[0]

In [ ]:
y_train.index[-1]

In [ ]:
order_aa = get_parametes.get('order')
seasonal_order_aa = get_parametes.get('seasonal_order')
        
model = SARIMAX(y_train,
                     order = (order_aa[0], order_aa[1], order_aa[2]),
                     seasonal_order =(seasonal_order_aa[0],
                                      seasonal_order_aa[1], seasonal_order_aa[2], seasonal_order_aa[3]))
result = model.fit(disp=0)
summary =result.summary()
summary

In [ ]:
fc= result.forecast()  # 95% conf
# Make as pandas series
fc_series = pd.Series(fc, index=y_test.index)
# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(y_train, label='training data')
plt.plot(y_test, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.title('ARCH CAPITAL GROUP Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('ARCH CAPITAL GROUP Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
order_aa = get_parametes.get('order')
seasonal_order_aa = get_parametes.get('seasonal_order')
        
model_ar = ARIMA(y_train,
                     order = (order_aa[0], order_aa[1], order_aa[2]))
result_ar = model.fit()
summary_ar =result.summary()
summary_ar

In [ ]:
# Forecast
Forecast= result.get_forecast(len(y_test)+10)  # 95% conf
conf = Forecast.conf_int()
# Make as pandas series
fc_series = pd.Series(fc, index=y_test.index)
lower_series = pd.Series(conf.loc[:, 'lower close'], index=y_test.index)
upper_series = pd.Series(conf.loc[:, 'upper close'], index=y_test.index)
# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(y_train, label='training data')
plt.plot(y_test, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('ARCH CAPITAL GROUP Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('ARCH CAPITAL GROUP Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
lower_series = pd.Series(conf.loc[:, 'lower close'], index=y_test.index)
lower_series

In [ ]:
Forecast.conf_int()

In [ ]:
result.conf_int()

In [ ]:
y_test.index

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = ARIMA(history, order = (1, 1, 0)).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])

In [ ]:
df_predictions_1 = pd.DataFrame({"y_test" : y_test, "y_pred_SAR" : y_pred_SAR})
fig = px.line(df_predictions_1)
fig.show()

In [ ]:
y_pred_ar = result.predict(start = len(y_train), end = (len(y_train)+len(y_test)+10) ,dynamic=False)
df_predictions = pd.DataFrame({"train" : y_train, "test" : y_test, "predict" : y_pred_ar})
fig = px.line(df_predictions)
fig

In [ ]:
y_pred_ar.conf_int()